In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from keras import models, layers, optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
import os
os.listdir('../input/scene-classification')

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
print(os.listdir("../input/scene-classification/"))

TRAIN_DIR = '../input/scene-classification/train-scene classification/train/'
# TEST_DIR = '../input/test_WyRytb0.csv/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

y_test = pd.read_csv("../input/scene-classification/test_WyRytb0.csv")
y_train = pd.read_csv("../input/scene-classification/train-scene classification/train.csv")
y_train = y_train['label']
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, 6)

train_images_list = []
test_images_list = []
a = list(y_test['image_name'])
for i in train_images:
    if str(i[63:]) not in a:
        train_images_list.append(i)
    if str(i[63:]) in a:
        test_images_list.append(i)
        
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_images_list.sort(key=natural_keys)
test_images_list.sort(key=natural_keys)

def prepare_data(list_of_images):
    x = [] 
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (150,150), interpolation=cv2.INTER_CUBIC))     
    return x
X_train = prepare_data(train_images_list)
X_test = prepare_data(test_images_list)

In [ ]:
X_train_prime, X_val, y_train_prime, y_val = train_test_split(X_train, y_train_categorical, random_state = 6, train_size = .80)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(np.array(X_train_prime), y_train_prime, batch_size = 32, shuffle = True)
val_generator = val_datagen.flow(np.array(X_val), y_val, batch_size = 32, shuffle = True)

In [ ]:
#import inception with pre-trained weights. do not include fully #connected layers
Xception_base = applications.resnet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = Xception_base.output￼
x = layers.GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(.5)(x)
# and a fully connected output/classification layer
predictions = layers.Dense(6, activation='softmax')(x)
# create the full network so we can train on it
Xception_transfer = models.Model(inputs=Xception_base.input, outputs=predictions)

In [ ]:
Xception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
              metrics=['accuracy'])


In [ ]:
history = Xception_transfer.fit_generator(
                    train_generator,
                    epochs=50,
                    steps_per_epoch=1500,
                    validation_data = val_generator,
                    validation_steps = 300,
                    workers = 4)

In [ ]:
history = Xception_transfer.fit_generator(
                    train_generator,
                    epochs=30,
                    steps_per_epoch=1500,
                    validation_data = val_generator,
                    validation_steps = 300,
                    workers = 4)